In [6]:
import sys
sys.path.append('../..')
from src import WindSpeedUnit, ForecastAPI, ForecastCurrent
import src

f = ForecastAPI()
f.latitude, f.longitude = src.GEO_COORDINATES.DUNDALK_IT.value
f.wind_speed_unit = WindSpeedUnit.METERS_PER_SECOND
f.current = [ForecastCurrent.WIND_DIRECTION_10M]

f.request()

Remaining: {'daily': 9990.0, 'hourly': 4998.0, 'minutely': 600}
API counts: 1.0


{'latitude': 53.990437,
 'longitude': -6.4023438,
 'generationtime_ms': 0.20265579223632812,
 'utc_offset_seconds': 0,
 'timezone': 'GMT',
 'timezone_abbreviation': 'GMT',
 'elevation': 15.0,
 'current_units': {'time': 'iso8601',
  'interval': 'seconds',
  'wind_direction_10m': '°'},
 'current': {'time': '2025-02-21T15:30',
  'interval': 900,
  'wind_direction_10m': 218}}

In [7]:
import pandas as pd
pd.read_sql_query('SELECT * FROM REQUESTS', f._conn) 

,id,url,call_weight,timestamp
0,1,https://api.open-meteo.com/v1/forecast?current...,1.0,2025-02-20 19:03:04
1,2,https://api.open-meteo.com/v1/forecast?current...,1.0,2025-02-20 19:03:42
2,3,https://api.open-meteo.com/v1/forecast?current...,1.0,2025-02-20 19:06:22
3,4,https://api.open-meteo.com/v1/forecast?current...,1.0,2025-02-20 19:09:55
4,5,https://api.open-meteo.com/v1/forecast?current...,1.0,2025-02-20 19:31:44
5,6,https://archive-api.open-meteo.com/v1/archive?...,1.0,2025-02-21 13:49:00
6,7,https://api.open-meteo.com/v1/forecast?current...,1.0,2025-02-21 14:01:14
7,8,https://archive-api.open-meteo.com/v1/archive?...,1.0,2025-02-21 14:10:11
8,9,https://archive-api.open-meteo.com/v1/archive?...,1.0,2025-02-21 14:52:31
9,10,https://archive-api.open-meteo.com/v1/archive?...,1.0,2025-02-21 14:55:21


In [8]:
src.weather.utils.RequestLogger.queryRemaining(f._conn)

{'daily': 9989.0, 'hourly': 4997.0, 'minutely': 599.0}

In [9]:
url = 'https://archive-api.open-meteo.com/v1/archive?latitude=52.0&longitude=12.0&latitude=52.52&longitude=13.41&start_date=2000-02-17&end_date=2025-02-19&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,boundary_layer_height,wet_bulb_temperature_2m,total_column_integrated_water_vapour,is_day,sunshine_duration,albedo,snow_depth_water_equivalent&daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&wind_speed_unit=ms&timezone=auto'
src.weather.utils.ApiCounter.calculate_call_weight_from_url(url) #3718.842 is the answer. It's wrong though.

7437.685714285714

In [10]:
parsedParams = src.weather.utils.ApiCounter.parse_url_params(url)

In [11]:
import urllib.parse

parsed_url = urllib.parse.urlparse(url)
query_params = urllib.parse.parse_qs(parsed_url.query)

query_params

{'latitude': ['52.0', '52.52'],
 'longitude': ['12.0', '13.41'],
 'start_date': ['2000-02-17'],
 'end_date': ['2025-02-19'],
 'hourly': ['temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,boundary_layer_height,wet_bulb_temperature_2m,total_column_integrated_water_vapour,is_day,sunshine_duration,albedo,snow_depth_water_equivalent'],
 'daily': ['weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_m

In [12]:
'''
https://archive-api.open-meteo.com/v1/archive
    ?latitude=52.52
    &longitude=13.41
    &start_date=2000-02-17
    &end_date=2025-02-19
    &hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,boundary_layer_height,wet_bulb_temperature_2m,total_column_integrated_water_vapour,is_day,sunshine_duration,albedo,snow_depth_water_equivalent
    &daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration
    &wind_speed_unit=ms
    &timezone=auto
'''.replace('\n', '').replace(' ', '').replace('\t', '')

'https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2000-02-17&end_date=2025-02-19&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,boundary_layer_height,wet_bulb_temperature_2m,total_column_integrated_water_vapour,is_day,sunshine_duration,albedo,snow_depth_water_equivalent&daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,daylight_duration,sunshine_duration,precipitation_sum,rain_sum,snowfall_sum,precipitation_hours,wind_speed_10m_max,wind_gusts_10m_max,wind_direction_10m_dominant,shortwave_radiation_sum,et0_fao_evapotranspiration&wind_speed_unit=ms&timezone=auto'


'https://archive-api.open-meteo.com/v1/archive?latitude=52.52&longitude=13.41&start_date=2000-02-17&end_date=2025-02-19&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,boundary_layer_height,wet_bulb_temperature_2m,total_column_integrated_water_vapour,is_day,sunshine_duration,albedo,snow_depth_water_equivalent&daily=weather_code,temperature_2m_max,temperature_2m_min,temperature_2m_mean,apparent_temperature_max,apparent_temperature_min,apparent_temperature_mean,sunrise,sunset,d

In [13]:
from src import WindSpeedUnit, HistoricalHourly, HistoricalAPI, HistoricalDaily
import datetime as dt

f = HistoricalAPI()
dundalk_it = (53.98352, -6.391390) #https://data.mendeley.com/datasets/tm988rs48k/2
f.latitude, f.longitude = dundalk_it
f.wind_speed_unit = WindSpeedUnit.METERS_PER_SECOND
# f.current = [ForecastCurrent.WIND_SPEED_10M, ForecastCurrent.WIND_DIRECTION_10M, ForecastCurrent.TEMPERATURE_2M]
f.hourly = [HistoricalHourly.TEMPERATURE_2M,
            HistoricalHourly.RELATIVE_HUMIDITY_2M, 
            HistoricalHourly.PRECIPITATION,
            HistoricalHourly.WIND_SPEED_10M,
            HistoricalHourly.WIND_DIRECTION_10M,
            HistoricalHourly.WIND_SPEED_100M,
            HistoricalHourly.WIND_DIRECTION_100M]
f.daily = [HistoricalDaily.TEMPERATURE_2M_MAX,]
f.start_date = dt.datetime(2019, 1, 1)
f.end_date = dt.datetime(2019, 1, 4) 

f.build_url()
result = f.request()


Remaining: {'daily': 9989.0, 'hourly': 4997.0, 'minutely': 599.0}
API counts: 1.0


In [14]:
for k,v in result.items():
    print(k,end = '')
    if isinstance(v, list):
        print('',v[:5],end='')
    elif isinstance(v, dict):
        print('',list(v.keys()),end='')
    else:
        print('',v,end='')
    print()

latitude 53.954304
longitude -6.4410095
generationtime_ms 78.38106155395508
utc_offset_seconds 0
timezone GMT
timezone_abbreviation GMT
elevation 15.0
hourly_units ['time', 'temperature_2m', 'relative_humidity_2m', 'precipitation', 'wind_speed_10m', 'wind_direction_10m', 'wind_speed_100m', 'wind_direction_100m']
hourly ['time', 'temperature_2m', 'relative_humidity_2m', 'precipitation', 'wind_speed_10m', 'wind_direction_10m', 'wind_speed_100m', 'wind_direction_100m']
daily_units ['time', 'temperature_2m_max']
daily ['time', 'temperature_2m_max']


In [27]:
import pandas as pd
import sqlite3
from copy import deepcopy

metadata = dict()
response = deepcopy(result)

for k,v in result.items():
    if not isinstance(v, (list,dict)):
        metadata[k] = response.pop(k)

dfHourly = pd.DataFrame(response['hourly'])
dfDaily = pd.DataFrame(response['daily'])
for k,v in metadata.items():
    if k == 'generationtime_ms':
        continue
    dfHourly[k] = v
    dfDaily[k] = v



In [28]:
dfDaily

,time,temperature_2m_max,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation
0,2019-01-01,9.6,53.954304,-6.441009,0,GMT,GMT,15.0
1,2019-01-02,7.6,53.954304,-6.441009,0,GMT,GMT,15.0
2,2019-01-03,7.3,53.954304,-6.441009,0,GMT,GMT,15.0
3,2019-01-04,6.4,53.954304,-6.441009,0,GMT,GMT,15.0


In [29]:
dfHourly

,time,temperature_2m,relative_humidity_2m,precipitation,wind_speed_10m,wind_direction_10m,wind_speed_100m,wind_direction_100m,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation
0,2019-01-01T00:00,8.9,85,0.0,4.49,258,7.99,261,53.954304,-6.441009,0,GMT,GMT,15.0
1,2019-01-01T01:00,9.1,84,0.0,4.57,260,7.95,264,53.954304,-6.441009,0,GMT,GMT,15.0
2,2019-01-01T02:00,9.1,86,0.0,4.63,264,8.22,267,53.954304,-6.441009,0,GMT,GMT,15.0
3,2019-01-01T03:00,9.2,86,0.0,4.62,265,8.10,269,53.954304,-6.441009,0,GMT,GMT,15.0
4,2019-01-01T04:00,9.1,87,0.0,4.40,267,7.90,271,53.954304,-6.441009,0,GMT,GMT,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,2019-01-04T19:00,1.9,90,0.0,2.12,172,3.81,177,53.954304,-6.441009,0,GMT,GMT,15.0
92,2019-01-04T20:00,2.0,90,0.0,1.80,177,4.30,177,53.954304,-6.441009,0,GMT,GMT,15.0
93,2019-01-04T21:00,1.1,92,0.0,2.55,191,4.10,183,53.954304,-6.441009,0,GMT,GMT,15.0
94,2019-01-04T22:00,1.0,92,0.0,2.46,207,4.00,193,53.954304,-6.441009,0,GMT,GMT,15.0


In [35]:
df.to_pickle('historical.pkl')

In [6]:
import pandas as pd
import sys
sys.path.append('../..')
import src

Raw_df = pd.read_pickle('historical.pkl')
df = Raw_df.copy()
df.time = pd.to_datetime(df.time,format = '%Y-%m-%dT%H:%M').astype('str')

# df.to_csv(src.SOURCE.DATA.str+'//historical_weather_data.csv',index = False)


In [ ]:
import sqlite3

'''
time                       {'str': 220345}
temperature_2m           {'float': 220345}
relative_humidity_2m     {'float': 220345}
precipitation            {'float': 220345}
wind_speed_10m           {'float': 220345}
wind_direction_10m       {'float': 220345}
wind_speed_100m          {'float': 220345}
wind_direction_100m      {'float': 220345}
latitude                 {'float': 220345}
longitude                {'float': 220345}
utc_offset_seconds         {'int': 220345}
timezone                   {'str': 220345}
timezone_abbreviation      {'str': 220345}
elevation                {'float': 220345}
'''
# df = Raw_df.copy()
# df.time = pd.to_datetime(df.time,format = '%Y-%m-%dT%H:%M').astype('str')
# df = df.melt(id_vars = 'time latitude longitude utc_offset_seconds timezone timezone_abbreviation elevation'.split(),
#         value_vars = 'temperature_2m relative_humidity_2m precipitation wind_speed_10m wind_direction_10m wind_speed_100m wind_direction_100m'.split(),
#         var_name = 'parameter',
#         value_name = 'value')
# make sqlite file with the above fields and datatypes. Table name is historical weather data
conn = sqlite3.connect(src.SOURCE.DATA.DB.str)

cursor = conn.cursor()
# enable Write Ahead Logging (WAL) mode
cursor.execute("PRAGMA journal_mode=WAL;")

# make if not exists
# COMMAND = '''
# CREATE TABLE IF NOT EXISTS historical_weather_data (
#     ID INTEGER PRIMARY KEY AUTOINCREMENT,
#     time TEXT,
#     temperature_2m REAL,
#     relative_humidity_2m REAL,
#     precipitation REAL,
#     wind_speed_10m REAL,
#     wind_direction_10m REAL,
#     wind_speed_100m REAL,
#     wind_direction_100m REAL,
#     latitude REAL,
#     longitude REAL,
#     utc_offset_seconds INTEGER,
#     timezone TEXT,
#     timezone_abbreviation TEXT,
#     elevation REAL
# );
# '''
COMMAND = '''
CREATE TABLE IF NOT EXISTS hourly_historical_weather_data(
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    time TEXT,
    latitude REAL,
    longitude REAL,
    utc_offset_seconds INTEGER,
    timezone TEXT,
    timezone_abbreviation TEXT,
    elevation REAL
    parameter TEXT,
    value REAL
);
'''

cursor.execute(COMMAND)
conn.commit()


: 

In [ ]:
# make sqlite file with the above fields and datatypes. Table name is historical weather data
conn = sqlite3.connect(src.SOURCE.DATA.DB.str)

cursor = conn.cursor()
# enable Write Ahead Logging (WAL) mode
cursor.execute("PRAGMA journal_mode=WAL;")

# make if not exists
# COMMAND = '''
# CREATE TABLE IF NOT EXISTS historical_weather_data (
#     ID INTEGER PRIMARY KEY AUTOINCREMENT,
#     time TEXT,
#     temperature_2m REAL,
#     relative_humidity_2m REAL,
#     precipitation REAL,
#     wind_speed_10m REAL,
#     wind_direction_10m REAL,
#     wind_speed_100m REAL,
#     wind_direction_100m REAL,
#     latitude REAL,
#     longitude REAL,
#     utc_offset_seconds INTEGER,
#     timezone TEXT,
#     timezone_abbreviation TEXT,
#     elevation REAL
# );
# '''
COMMAND = '''
CREATE TABLE IF NOT EXISTS daily_historical_weather_data(
    ID INTEGER PRIMARY KEY AUTOINCREMENT,
    time TEXT,
    latitude REAL,
    longitude REAL,
    utc_offset_seconds INTEGER,
    timezone TEXT,
    timezone_abbreviation TEXT,
    elevation REAL,
    parameter TEXT,
    value REAL
);
'''
# cursor.execute('DROP TABLE IF EXISTS daily_historical_weather_data')
cursor.execute(COMMAND)
conn.commit()


In [3]:
# # df.to_sql('hourly_historical_weather_data', conn, if_exists='replace', index = False)
# dfDaily.melt(id_vars = 'time latitude longitude utc_offset_seconds timezone timezone_abbreviation elevation'.split(),
#         var_name = 'parameter',
#         value_name = 'value')

import pandas as pd
import sqlite3
import sys

sys.path.append('../..')
import src

conn = sqlite3.connect(src.SOURCE.DATA.DB.str)
pd.read_sql_query('SELECT * FROM daily_historical_weather_data', conn)#.merge(
#     dfDaily.melt(id_vars = 'time latitude longitude utc_offset_seconds timezone timezone_abbreviation elevation'.split(),
#         var_name = 'parameter',
#         value_name = 'value')
# )

,ID,time,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation,value


In [34]:
pd.read_pickle('historical.pkl').melt(id_vars = 'time latitude longitude utc_offset_seconds timezone timezone_abbreviation elevation'.split(),
        var_name = 'parameter',
        value_name = 'value')

,time,latitude,longitude,utc_offset_seconds,timezone,timezone_abbreviation,elevation,parameter,value
0,2000-01-01T00:00,53.954304,-6.441009,0,GMT,GMT,15.0,temperature_2m,4.8
1,2000-01-01T01:00,53.954304,-6.441009,0,GMT,GMT,15.0,temperature_2m,3.9
2,2000-01-01T02:00,53.954304,-6.441009,0,GMT,GMT,15.0,temperature_2m,3.4
3,2000-01-01T03:00,53.954304,-6.441009,0,GMT,GMT,15.0,temperature_2m,2.8
4,2000-01-01T04:00,53.954304,-6.441009,0,GMT,GMT,15.0,temperature_2m,2.2
...,...,...,...,...,...,...,...,...,...
1542410,2025-02-18T20:00,53.954304,-6.441009,0,GMT,GMT,15.0,wind_direction_100m,131.0
1542411,2025-02-18T21:00,53.954304,-6.441009,0,GMT,GMT,15.0,wind_direction_100m,132.0
1542412,2025-02-18T22:00,53.954304,-6.441009,0,GMT,GMT,15.0,wind_direction_100m,130.0
1542413,2025-02-18T23:00,53.954304,-6.441009,0,GMT,GMT,15.0,wind_direction_100m,131.0
